In [ ]:
import pickle
import boto3
import re
import json
import random
import unicodedata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType, FloatType, ArrayType, DoubleType, StructType, StructField,LongType

In [ ]:
save_path = # path for saving

In [ ]:
@udf(returnType=ArrayType(StructType([StructField('keyword', StringType()), 
                                     StructField('score', FloatType())])))
def get_json_from_list(list_of_keywords):                            
    final_json = []
    try:
        for keyword in list_of_keywords:
            check_keyword = check_for_non_latin_characters(keyword[0])
            if check_keyword:
                final_json.append({'keyword': keyword[0], 'score': float(keyword[1])})
    except:
        pass

    return final_json

### Storing keywords

In [ ]:
# mapping of work_id to original_title
works = spark.read.parquet(f"{save_path}/work_titles/") \
    .select(F.col('paper_id').alias('work_id'), 'original_title') \
    .dropDuplicates(subset=['work_id'])

# deduplicated titles
raw_data = spark.read.parquet(f"{save_path}/work_titles/") \
    .select('original_title').withColumn('paper_title', F.trim(F.col('original_title'))) \
    .dropDuplicates(subset=['original_title'])


# keywords for each deduplicated title
processed_data = spark.read.parquet(f"{save_path}/all_keywords/")


In [ ]:
# join raw data to newly generated keywords
raw_data.join(processed_data.select('paper_title','keywords'), how='left', on='paper_title') \
    .write.mode('overwrite') \
    .parquet(f"{save_path}/original_titles_to_keywords/")

In [ ]:
keywords = spark.read.parquet(f"{save_path}/original_titles_to_keywords/")

In [ ]:
works.join(keywords, how='left', on='original_title').select('work_id', 'keywords') \
    .withColumn('json_struct_col', get_json_from_list(F.col('keywords'))) \
    .withColumn('json_col', F.to_json(F.col('json_struct_col'))) \
    .select('work_id', 'keywords', 'json_col') \
    .write.mode('overwrite') \
    .parquet(f"{save_path}/data_for_table/")

In [ ]:
secret = # secrets for database

spark.read.parquet(f"{save_path}/data_for_table/") \
    .select('work_id', F.col('json_col').alias('keywords')) \
    .withColumn("created", F.current_timestamp()) \
    .withColumn("updated", F.current_timestamp()) \
    .coalesce(6) \
    .write.format("jdbc") \
    .option("url", f"jdbc:postgresql://{secret['host']}:{secret['port']}/{secret['dbname']}") \
    .option("dbtable", 'keywords_table') \
    .option("user", secret['username']) \
    .option("password", secret['password']) \
    .option("driver", "org.postgresql.Driver") \
    .option("stringtype", "unspecified") \
    .option("numPartitions", 6) \
    .mode("append") \
    .save()